In [2]:
import pandas as pd
import json

# Data Ingestion and Indexing

In [4]:
df=pd.read_json('../data/recipes.json')

In [5]:
receipes=df.to_dict(orient='records')

In [8]:
print(len(receipes))

360


In [11]:
receipes[2]


{'title': 'Turmeric Flatbread',
 'date': Timestamp('2022-08-09 00:00:00'),
 'tags': ['bread', 'turkish'],
 'introduction': 'A great companion to Turkish Red Lentil Soup. - ⏲️ Prep time: 20 min - 🍳 Cook time: 10 min - 🍽️ Servings: 4',
 'ingredients': '- 1 cup Wheat Flour (white or whole) - 1/2 Tbsp Ground Turmeric - 1/2 tsp Kosher Salt - 1/4 tsp Baking Powder - 2 Tbsp Olive Oil',
 'direction': '1. Combine flour, turmeric, salt and baking powder.  2. Stir in 1/3 cup water and the olive oil.  3. Knead until smooth (1-2 min). Cover and set aside 15 minutes.  4. Cut the dough into 4 pieces; flatten each into a 6-inch round.  5. Cook one at a time in an oiled skillet over medium-high heat, flip when browned (2 mins).',
 'output': {'title': 'Turmeric Flatbread',
  'ingredients': '- 1 cup Wheat Flour (white or whole) - 1/2 Tbsp Ground Turmeric - 1/2 tsp Kosher Salt - 1/4 tsp Baking Powder - 2 Tbsp Olive Oil',
  'direction': '1. Combine flour, turmeric, salt and baking powder.  2. Stir in 1/3 c

In [22]:
from sentence_transformers import SentenceTransformer
model_name='all-MiniLM-L12-v2'
model=SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
from elasticsearch import Elasticsearch

es_client=Elasticsearch('http://localhost:9200')

index_settings={
    "settings":{
            "number_of_shards":1,
            "number_of_replicas":0
    },
    "mappings":{
            "properties":{
                    "title":{"type":"text"},
                    "tags":{"type":"text"},
                    "introduction":{"type":"text"},
                    "ingredients":{"type":"text"},
                    "direction":{"type":"text"},
                    "combined_vector":{
                            "type":"dense_vector",
                            "dims":384,
                            "index":True,
                            "similarity":"cosine"
                     },
            },
    }
}
index_name="food_recipes"
es_client.indices.delete(index=index_name,ignore_unavailable=True)
es_client.indices.create(index=index_name,body=index_settings)                
                    
                        
    
    

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'food_recipes'})

In [29]:
from tqdm.auto import tqdm

for recipe in tqdm(receipes):
            title = recipe['title']
            ingredients = recipe['ingredients']
            direction = recipe['direction']
            combined_vector=model.encode(title + ' '+ ingredients + ' '+ direction )
            try:
                es_client.index(index=index_name,document=recipe)  
            except Exception as e:
                print(e)
            
 

  0%|          | 0/360 [00:00<?, ?it/s]

In [38]:
def elastic_search_knn_query(field,vector):
            knn={
                "field":field,
                "query_vector":vector,
                "k":5,
                "num_candidates": 360,
            }
            search_query={
                "knn":knn,
                "_source":["title","tags","introduction","ingredients","directions","output"]
            }
            es_results=es_client.search(index=index_name,body=search_query)
            results=[]
            for hit in es_results['hits']['hits']:
                    results.append(hit['_source'])
            return results
            

In [39]:
def vector_query(q):
        question=q['question']
        vect_query=model.encode(question)
        return elastic_search_knn_query('combined_vector',vect_query)
            

In [40]:
vector_query(dict(
    question='how to make lentil soup?'
    ))

[]